In [1]:
from src.features.build_features import handle_zeros, KL_apply
from src.visualization.visualize import plot_divergence
from src.data.make_dataset import preprocess
from pathlib import Path
import pandas as pd
import numpy as np


In [2]:
FILE_NAME = 'uniform_date.csv'
PATH = Path('../data/raw')
cycle3 = pd.read_csv(PATH / FILE_NAME)
processed = preprocess(cycle3)

/Users/zarak/.local/share/virtualenvs/focus_plots-vLdWlo3v/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [3]:
processed.shape

(11351, 18)

In [14]:
processed = handle_zeros(processed)

> For the group of timestamps that you are considering for each bubble, we want to calculate the mean for each bin mu_b where b is the bin. Then for each forecast f_t_b, where t is the index of the timestamp, we are going to calculate the KL-divergence
K_t = sum(mu_b * log2(f_t_b / mu_b)) along the b axis
Finally we are going to calculate the mean forKL, mean(KL_t) along the t axis.

**So we will have to compute means for the group, during that interval, then we will compute the divergences from that mean for each forecast in the time interval, and then we will average the divergences.**

In [4]:
processed.head(3)

,Forecaster,Forecaster.ID,Question,Forecast,TeamName,Commuity.ID,Ordered.Bin.Number,Number.of.Bins,True.Answer,Weight,WRPS,CrossEntropy,FairSkill,SWRPS,Final,Initial,Process.Gain,Tvalue
Uniform Date Format,,,,,,,,,,,,,,,,,,
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.7,Kiwi,1,1,2,0.60,1.0,0.01,-0.695,-0.002,0.10,0,1,0.15,43780.91706
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.3,Kiwi,1,2,2,0.40,1.0,0.01,-0.695,-0.002,0.10,0,1,0.15,43780.91706
2019-11-11 22:02:20,Dimitry Sarin,71,CFF 308.1,0.3,Kiwi,1,2,2,0.93,1.0,0.3969,-1.143,-0.450,0.63,0,1,-0.37,43780.91829


- The KL-divergence I had in mind is different. First we compute the average of the forecasts in that time window, then we compute for each forecaster the divergence of that forecaster from that time window average. I also prefer to vectorize that operation as I did in DTC, because I get a feel for the dimensions involved.
- vector{Ave_i} dot.product ( log2( {matrix forecasters of shape (m,n)} / vector{Ave_i} ) )
- The division is done by bin with broadcast to all n forecasters. Then when you have all the KL_divergences for each forecaster, you average them to obtain the bubble size for that time window.
- The number you calculated, I believe is sort of an entropy for the average.


Notes:
1. $m$ is the number of bins
2. $n$ is the number of forecasters
3. $i$ is the index of the timestamp

In [5]:
def kullback_leibler(forecasts, average):
    print("forecasts", forecasts)
    print("average", average)
    return -average @ (np.log2(forecasts) - np.log2(average.T))

In [6]:
def test_kullback_leibler():
    forecasts = np.array(
        [[0.1, 0.1],
         [0.9, 0.9]]
    )
    average = np.array([[0.1, 0.9]])
    print((kullback_leibler(forecasts, average) == 0).all())

In [7]:
test_kullback_leibler()

forecasts [[0.1 0.1]
 [0.9 0.9]]
average [[0.1 0.9]]
True


In [8]:
def KL_apply(question):
    if not question.empty:
        average_forecast = question.groupby(
            ['Ordered.Bin.Number']
        ).Forecast.mean().reset_index()
        forecasts = question.pivot_table(
            values='Forecast',
            index='Ordered.Bin.Number',
            columns='Forecaster.ID'
        ).values
        question_avg = average_forecast.Forecast.values.reshape(1, -1)
        divergences = kullback_leibler(forecasts, question_avg)
        print('divs', divergences)
        return np.mean(divergences)

In [9]:
processed.Question.unique()

array(['CFF 307.1', 'CFF 308.1', 'CFF 308.3', 'CFF 308.4', 'CFF 307.3',
       'CFF 307.4', 'CFF 307.2', 'CFF 301.1', 'CFF 301.2', 'CFF 301.4',
       'CFF 301.3', 'CFF 304.1', 'CFF 304.2', 'CFF 304.4', 'CFF 304.3',
       'CFF 308.2'], dtype=object)

In [404]:
processed.head(2)

,Forecaster,Forecaster.ID,Question,Forecast,TeamName,Commuity.ID,Ordered.Bin.Number,Number.of.Bins,True.Answer,Weight,WRPS,CrossEntropy,FairSkill,SWRPS,Final,Initial,Process.Gain,Tvalue
Uniform Date Format,,,,,,,,,,,,,,,,,,
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.7,Kiwi,1,1,2,0.6,1.0,0.01,-0.695,-0.002,0.1,0,1,0.15,43780.91706
2019-11-11 22:00:34,Dimitry Sarin,71,CFF 307.1,0.3,Kiwi,1,2,2,0.4,1.0,0.01,-0.695,-0.002,0.1,0,1,0.15,43780.91706


In [24]:
question = 'CFF 307.3'
resampling_period = 'H'

In [25]:
res = processed.query("Question == @question").reset_index().groupby(pd.Grouper(
    freq=resampling_period,
    label='right',
    key='Uniform Date Format',
)).apply(KL_apply)

forecasts [[0.0049505 ]
 [0.0049505 ]
 [0.03960396]
 [0.45544554]
 [0.4950495 ]]
average [[0.0049505  0.0049505  0.03960396 0.45544554 0.4950495 ]]
divs [[0.]]
forecasts [[0.02]
 [0.02]
 [0.18]
 [0.37]
 [0.41]]
average [[0.02 0.02 0.18 0.37 0.41]]
divs [[0.]]
forecasts [[0.05]
 [0.25]
 [0.3 ]
 [0.25]
 [0.15]]
average [[0.05 0.25 0.3  0.25 0.15]]
divs [[0.]]
forecasts [[0.00497512]
 [0.00995025]
 [0.039801  ]
 [0.45771144]
 [0.48756219]]
average [[0.00497512 0.00995025 0.039801   0.45771144 0.48756219]]
divs [[0.]]
forecasts [[0.02]
 [0.04]
 [0.1 ]
 [0.44]
 [0.4 ]]
average [[0.02 0.04 0.1  0.44 0.4 ]]
divs [[0.]]
forecasts [[0.00497512 0.00497512]
 [0.04975124 0.04975124]
 [0.14925373 0.69651741]
 [0.74626866 0.19900498]
 [0.04975124 0.04975124]]
average [[0.00497512 0.04975124 0.42288557 0.47263682 0.04975124]]
divs [[0.3239355  0.28538451]]
forecasts [[0.0049505 ]
 [0.0049505 ]
 [0.07920792]
 [0.71617162]
 [0.19471947]]
average [[0.0049505  0.0049505  0.07920792 0.71617162 0.19471947]

forecasts [[0.05]
 [0.08]
 [0.17]
 [0.53]
 [0.17]]
average [[0.05 0.08 0.17 0.53 0.17]]
divs [[0.]]
forecasts [[0.00497512]
 [0.00995025]
 [0.20232172]
 [0.66334992]
 [0.11940299]]
average [[0.00497512 0.00995025 0.20232172 0.66334992 0.11940299]]
divs [[0.]]
forecasts [[0.00497512]
 [0.04975124]
 [0.19900498]
 [0.58706468]
 [0.15920398]]
average [[0.00497512 0.04975124 0.19900498 0.58706468 0.15920398]]
divs [[0.]]
forecasts [[0.00497512]
 [0.09950249]
 [0.34825871]
 [0.39800995]
 [0.14925373]]
average [[0.00497512 0.09950249 0.34825871 0.39800995 0.14925373]]
divs [[0.]]
forecasts [[0.0049505 ]
 [0.0049505 ]
 [0.04950495]
 [0.34653465]
 [0.59405941]]
average [[0.0049505  0.0049505  0.04950495 0.34653465 0.59405941]]
divs [[0.]]
forecasts [[0.00748756]
 [0.01995025]
 [0.0599005 ]
 [0.36905473]
 [0.54360697]]
average [[0.00748756 0.01995025 0.0599005  0.36905473 0.54360697]]
divs [[0.]]
forecasts [[0.00497512]
 [0.09950249]
 [0.29850746]
 [0.39800995]
 [0.19900498]]
average [[0.0049751

In [56]:
res[res < 0].index

DatetimeIndex(['2019-11-25 03:00:00'], dtype='datetime64[ns]', name='Uniform Date Format', freq=None)

In [61]:
import datetime

In [67]:
datetime.datetime.fromtimestamp('2019-11-25')

TypeError: an integer is required (got type str)

In [87]:
processed.query("Question == 'CFF 307.3'").set_index('Uniform Date Format').between_time('02:31:50', '02:32:01').sort_values(by='Forecast').drop(columns='Forecaster')

,Forecaster.ID,Question,Forecast,TeamName,Commuity.ID,Ordered.Bin.Number,Number.of.Bins,True.Answer,Weight,WRPS,CrossEntropy,FairSkill,SWRPS,Final,Initial,Process.Gain,Tvalue,Total
Uniform Date Format,,,,,,,,,,,,,,,,,,
2019-11-25 02:32:00,63,CFF 307.3,0.002488,Mango,2,1,5,0.00,2.52,0.0316,-1.069,0.5400,0.1777,0,0,-0.342,43794.10556,2.01
2019-11-25 02:32:00,63,CFF 307.3,0.002488,Mango,2,1,5,0.00,2.52,0.0369,-1.075,0.5345,0.1921,1,0,-0.342,NaN,2.01
2019-11-25 02:32:00,63,CFF 307.3,0.004975,Mango,2,2,5,0.02,2.52,0.0316,-1.069,0.5400,0.1777,0,0,-0.342,43794.10556,2.01
2019-11-25 02:32:00,63,CFF 307.3,0.014925,Mango,2,2,5,0.02,2.52,0.0369,-1.075,0.5345,0.1921,1,0,-0.342,NaN,2.01
2019-11-25 02:32:00,63,CFF 307.3,0.044776,Mango,2,3,5,0.07,2.52,0.0369,-1.075,0.5345,0.1921,1,0,-0.342,NaN,2.01
2019-11-25 02:32:00,63,CFF 307.3,0.059701,Mango,2,3,5,0.07,2.52,0.0316,-1.069,0.5400,0.1777,0,0,-0.342,43794.10556,2.01
2019-11-25 02:32:00,63,CFF 307.3,0.174129,Mango,2,4,5,0.62,2.52,0.0369,-1.075,0.5345,0.1921,1,0,-0.342,NaN,2.01
2019-11-25 02:32:00,63,CFF 307.3,0.179104,Mango,2,4,5,0.62,2.52,0.0316,-1.069,0.5400,0.1777,0,0,-0.342,43794.10556,2.01
2019-11-25 02:32:00,63,CFF 307.3,0.253731,Mango,2,5,5,0.29,2.52,0.0316,-1.069,0.5400,0.1777,0,0,-0.342,43794.10556,2.01


In [85]:
C = 0.005
processed.loc[:, 'Forecast'] = processed.Forecast.replace(0, C)
sums = processed.groupby(
    ['Uniform Date Format', 'Question']
).Forecast.sum().reset_index().rename(columns={'Forecast': 'Total'})

In [18]:
sums.head()

,Uniform Date Format,Question,Total
0,2019-11-11 22:00:34,CFF 307.1,1.0
1,2019-11-11 22:02:20,CFF 308.1,1.0
2,2019-11-11 22:02:59,CFF 308.3,1.0
3,2019-11-11 22:03:38,CFF 308.4,1.0
4,2019-11-11 22:06:03,CFF 307.3,1.0


In [102]:
processed.columns

Index(['Forecaster', 'Uniform Date Format', 'Forecaster.ID', 'Question',
       'Forecast', 'TeamName', 'Commuity.ID', 'Ordered.Bin.Number',
       'Number.of.Bins', 'True.Answer', 'Weight', 'WRPS', 'CrossEntropy',
       'FairSkill', 'SWRPS', 'Final', 'Initial', 'Process.Gain', 'Tvalue',
       'Total'],
      dtype='object')

In [22]:
def test_handle_zeros():
    forecasts = merged.Forecast
    grouped = merged.groupby(
        ['Uniform Date Format', 'Question']
    ).Forecast.sum().values
    assert all(np.logical_and(0 <= forecasts, forecasts <= 1))
    assert all(np.isclose(grouped, 1))

In [23]:
test_handle_zeros()

In [88]:
processed.shape

(11351, 20)

In [118]:
index_cols = ['Uniform Date Format', 'Question', 'Ordered.Bin.Number']
agg_dict = {col: 'last' for col in processed.columns if col not in index_cols}
processed.groupby(
    index_cols
, as_index=False).agg(agg_dict)

,Uniform Date Format,Question,Ordered.Bin.Number,Forecaster,Forecaster.ID,Forecast,TeamName,Commuity.ID,Number.of.Bins,True.Answer,Weight,WRPS,CrossEntropy,FairSkill,SWRPS,Final,Initial,Process.Gain,Tvalue,Total
0,2019-11-11 22:00:34,CFF 307.1,1,Dimitry Sarin,71,0.700000,Kiwi,1,2,0.60,1.00,0.01,-0.695,-0.0020,0.1000,0,1,0.1500,43780.91706,1.000
1,2019-11-11 22:00:34,CFF 307.1,2,Dimitry Sarin,71,0.300000,Kiwi,1,2,0.40,1.00,0.01,-0.695,-0.0020,0.1000,0,1,0.1500,43780.91706,1.000
2,2019-11-11 22:02:20,CFF 308.1,1,Dimitry Sarin,71,0.700000,Kiwi,1,2,0.07,1.00,0.3969,-1.143,-0.4500,0.6300,0,1,-0.3700,43780.91829,1.000
3,2019-11-11 22:02:20,CFF 308.1,2,Dimitry Sarin,71,0.300000,Kiwi,1,2,0.93,1.00,0.3969,-1.143,-0.4500,0.6300,0,1,-0.3700,43780.91829,1.000
4,2019-11-11 22:02:59,CFF 308.3,1,Dimitry Sarin,71,0.004950,Kiwi,1,5,0.01,2.52,0.1155,-1.599,0.0106,0.3398,0,1,-0.0770,43780.91874,1.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11283,2019-11-25 04:44:00,CFF 304.4,1,David Smerdon,87,0.004975,Mango,2,5,0.13,2.52,0.1143,-2.363,-0.7540,0.3381,1,0,0.2497,NaN,1.005
11284,2019-11-25 04:44:00,CFF 304.4,2,David Smerdon,87,0.049751,Mango,2,5,0.22,2.52,0.1143,-2.363,-0.7540,0.3381,1,0,0.2497,NaN,1.005
11285,2019-11-25 04:44:00,CFF 304.4,3,David Smerdon,87,0.348259,Mango,2,5,0.32,2.52,0.1143,-2.363,-0.7540,0.3381,1,0,0.2497,NaN,1.005
11286,2019-11-25 04:44:00,CFF 304.4,4,David Smerdon,87,0.447761,Mango,2,5,0.14,2.52,0.1143,-2.363,-0.7540,0.3381,1,0,0.2497,NaN,1.005


In [116]:
processed.shape

(11351, 20)

In [94]:
11351 - 11288

63

In [95]:
processed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11351 entries, 0 to 11350
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Forecaster           11351 non-null  object        
 1   Uniform Date Format  11351 non-null  datetime64[ns]
 2   Forecaster.ID        11351 non-null  int64         
 3   Question             11351 non-null  object        
 4   Forecast             11351 non-null  float64       
 5   TeamName             11351 non-null  object        
 6   Commuity.ID          11351 non-null  int64         
 7   Ordered.Bin.Number   11351 non-null  int64         
 8   Number.of.Bins       11351 non-null  int64         
 9   True.Answer          11351 non-null  float64       
 10  Weight               11351 non-null  float64       
 11  WRPS                 11351 non-null  object        
 12  CrossEntropy         11351 non-null  float64       
 13  FairSkill            11351 non-